In [1]:
#installing Necessary libraries
!pip install pyairtable python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.1/101.1 kB 6.2 MB/s eta 0:00:00


In [2]:
#importing necessary libraries
import os
from dotenv import load_dotenv # Necessary only if reading variables from a .env file
from pyairtable import Api, Table
import pandas as pd

# Here we include our base id and also the access token that we created from our Airtable
PAT = "YOUR_ACCESS_TOKEN(ensure it allows reading and writing of the table)" # Your PAT (Personal Access Token) is pasted here
BASE_ID = "The base ID. i.e:(App[****])" # Your Airtable Base ID is pasted here
TABLE_NAME = "Customers Table"

api = Api(PAT)
table = Table(PAT, BASE_ID, TABLE_NAME)

/tmp/ipython-input-3588040376.py:11: DeprecationWarning: Passing API keys or base IDs to pyairtable.Table is deprecated; use Api.table() or Base.table() instead. See https://pyairtable.rtfd.org/en/latest/migrations.html for details.
  table = Table(PAT, BASE_ID, TABLE_NAME)


In [6]:
# loading the data into our environment
rows = []
for rec in table.all():  # honors 5 rps; auto-retries on 429s
    fields = rec.get("fields", {})
    rows.append({
        "id": rec["id"],
        "CustomerID": fields.get("CustomerID"),
        "Gender": fields.get("Gender"),
        "Age": fields.get("Age"),
        "Annual Income (k$)": fields.get("Income (k$)"),
        "Spending Score (1-100)": fields.get("Spending Score"),
    })

df = pd.DataFrame(rows)

In [7]:
df.tail()

,id,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
195,recxruwGH0JdjQzoT,None,Female,23,72,99
196,recz1Fm0ncxXsu4DA,None,Male,56,77,45
197,reczIOrJRCvbNsR3W,None,Female,33,136,15
198,reczauhoW5fGmCUzA,None,Female,45,138,87
199,reczzDIsGTmQL5UMZ,None,Female,57,104,26


In [8]:
# create a new column for the preprocessing step
def high_value(row):
    try:
        return (row["Spending Score (1-100)"] >= 70) and (row["Annual Income (k$)"] >= 70)
    except TypeError:
        return False

df["Is High Value"] = df.apply(high_value, axis=1)
df.head()

,id,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100),Is High Value
0,rec04OyAHs7eneio5,None,Male,69,100,39,False
1,rec0Fhq5Ppzq1PSRD,None,Female,34,100,74,True
2,rec0dYwOzllMfi7EZ,None,Male,30,141,37,False
3,rec105hRqBOREKThX,None,Female,61,108,6,False
4,rec1qJYxLapM5ydua,None,Male,43,15,63,False


In [9]:
# After naming adding the column name and specifying the column category we can now add the data into it.
updates = []
for _, r in df.iterrows():
    if pd.isna(r["id"]):
        continue
    updates.append({
        "id": r["id"],
        "fields": {
            "High Value": bool(r["Is High Value"])
        }
    })

if updates:
    table.batch_update(updates)
